In [1]:
import pandas as pd
import calendar
import numpy as np

<h3>File selection - run either Training file or Test file</h3>

In [51]:
filetype1 = 'train.csv'
filetype2 = 'test.csv'

<h3>Training file</h3>

In [52]:
train = pd.read_csv("data/train.csv")
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [53]:
print(train["Upc"].nunique())
print(train["FinelineNumber"].nunique())
print(list(train["Upc"].value_counts())[:200])
print("####")
print(list(train["FinelineNumber"].value_counts())[:200])

97714
5195
[7657, 2501, 2067, 1883, 1652, 1515, 1487, 1405, 1370, 1343, 1018, 973, 953, 896, 856, 828, 815, 778, 778, 750, 745, 719, 698, 697, 686, 644, 622, 620, 618, 561, 560, 528, 526, 525, 515, 510, 502, 500, 495, 494, 486, 482, 455, 454, 435, 433, 427, 426, 425, 425, 425, 419, 419, 418, 413, 408, 407, 406, 405, 402, 400, 398, 398, 396, 396, 395, 384, 381, 381, 378, 376, 372, 370, 365, 364, 363, 360, 360, 357, 357, 356, 345, 343, 343, 338, 332, 328, 326, 324, 323, 316, 313, 311, 311, 309, 308, 304, 302, 301, 301, 293, 290, 285, 284, 283, 282, 279, 279, 279, 270, 268, 267, 266, 265, 263, 262, 259, 259, 259, 258, 251, 251, 245, 241, 241, 240, 238, 237, 236, 235, 235, 235, 233, 232, 232, 231, 230, 229, 229, 228, 227, 227, 226, 223, 223, 223, 222, 221, 221, 220, 220, 220, 219, 219, 218, 217, 217, 216, 215, 214, 214, 213, 213, 210, 208, 208, 206, 204, 203, 202, 202, 202, 200, 199, 199, 198, 198, 197, 197, 196, 196, 196, 195, 194, 193, 192, 191, 191, 190, 190, 189, 189, 189, 187, 187, 18

In [54]:
fname = 'data/'+filetype1
processed_fname = 'data/processed_data/processed_'+filetype1
print(fname, processed_fname)
df = pd.read_csv(fname)
df.head()
df_groups_train = df.groupby(df['DepartmentDescription']).groups
df_groups_train.keys();
freq_upc = df['Upc'].value_counts().keys()[:200]
freq_upc_features = ["UPC_"+str(upc) for upc in freq_upc]
freq_upc_features.extend(["INDICATOR_UPC_"+str(upc) for upc in freq_upc])
freq_upc_features;
freq_fineline = df['FinelineNumber'].value_counts().keys()[:200]
freq_fineline_features = ["FL_"+str(fl) for fl in freq_fineline]
freq_fineline_features.extend(["INDICATOR_FL_"+str(fl) for fl in freq_fineline]) 
freq_fineline_features;

data/train.csv data/processed_data/processed_train.csv


<h3>Test file</h3>

In [98]:
fname = 'data/'+filetype2
processed_fname = 'data/processed_data/processed_'+filetype2
print(fname, processed_fname)

data/test.csv data/processed_data/processed_test.csv


<h3>Data Processing</h3>

In [99]:
df = pd.read_csv(fname)
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [100]:
df.count()

VisitNumber              653646
Weekday                  653646
Upc                      649660
ScanCount                653646
DepartmentDescription    652318
FinelineNumber           649660
dtype: int64

<h3>Drop NA values</h3>

In [101]:
df = df.fillna(method="ffill")
df.count()

VisitNumber              653646
Weekday                  653646
Upc                      653646
ScanCount                653646
DepartmentDescription    653646
FinelineNumber           653646
dtype: int64

In [102]:
print(df['DepartmentDescription'].nunique())
#print(df['TripType'].nunique())
print(df['FinelineNumber'].groupby(df['DepartmentDescription']).nunique())

67
DepartmentDescription
1-HR PHOTO                     13
ACCESSORIES                    88
AUTOMOTIVE                    256
BAKERY                         93
BATH AND SHOWER                81
BEAUTY                        177
BEDDING                       186
BOOKS AND MAGAZINES            73
BOYS WEAR                     179
BRAS & SHAPEWEAR               59
CAMERAS AND SUPPLIES           29
CANDY, TOBACCO, COOKIES        70
CELEBRATION                   172
COMM BREAD                     51
CONCEPT STORES                  1
COOK AND DINE                 307
DAIRY                         108
DSD GROCERY                   362
ELECTRONICS                   163
FABRICS AND CRAFTS            340
FINANCIAL SERVICES             25
FROZEN FOODS                   94
FURNITURE                      32
GIRLS WEAR, 4-6X  AND 7-14    210
GROCERY DRY GOODS             403
HARDWARE                      324
HOME DECOR                    193
HOME MANAGEMENT               177
HORTICULTURE AND ACCESS

<h3>Add new feature FLCount corresponding to number of refined categories (FinelineNumbers) for each DepartmentDescription</h3>

In [103]:
df2 = pd.DataFrame(df['FinelineNumber'].groupby(df['DepartmentDescription']).nunique())
df2 = df2.rename(columns={'FinelineNumber': "FLCount"})
df2.head()

,FLCount
DepartmentDescription,
1-HR PHOTO,13
ACCESSORIES,88
AUTOMOTIVE,256
BAKERY,93
BATH AND SHOWER,81


In [104]:
df = df.join(df2, on='DepartmentDescription')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount
0,1,Friday,7.250339e+10,1,SHOES,3002.0,464
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,108
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,108
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,362


<h3>Add new feature VisitFLCount corresponding to number of refined categories (FinelineNumbers) for each VisitNumber</h3>

In [105]:
df2 = pd.DataFrame(df['FinelineNumber'].groupby(df['VisitNumber']).nunique())
df2 = df2.rename(columns={'FinelineNumber': "VisitFLCount"})
df2.head()

,VisitFLCount
VisitNumber,
1,4
2,3
3,1
4,1
6,1


In [106]:
df = df.join(df2, on='VisitNumber')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount
0,1,Friday,7.250339e+10,1,SHOES,3002.0,464,4
1,1,Friday,1.707711e+09,1,DAIRY,1526.0,108,4
2,1,Friday,8.947000e+10,1,DAIRY,1431.0,108,4
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0,362,3


<h3>Scalar encode weekday names for numerical feature</h3>

In [107]:
def encode_day_names(days):
    day_to_num = dict(zip(list(calendar.day_name), range(0, 7)))
    return [day_to_num[day] for day in days]

In [108]:
df['Weekday'] = encode_day_names(df['Weekday'])
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3


<h3>Add features corresponding to number of returns and number of purchases for a particular DepartmentDescription</h3>

In [109]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    #print(dept_desc)
    returns = 0
    purchases = 0
    for idx in ids:
        if df.loc[idx]['ScanCount'] < 0:
            returns += df.loc[idx]['ScanCount']
        else:
            purchases += df.loc[idx]['ScanCount']
    data.append({"DepartmentDescription":dept_desc, "NumReturns":returns, "NumPurchases":purchases})

In [110]:
df2 = pd.DataFrame(data)
df2['NumReturns'] = abs(df2['NumReturns'])
df2.head()

,DepartmentDescription,NumPurchases,NumReturns
0,1-HR PHOTO,500,40
1,ACCESSORIES,1427,91
2,AUTOMOTIVE,5932,388
3,BAKERY,8351,113
4,BATH AND SHOWER,5142,202


In [111]:
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848


<h3>Add features corresponding to number of returns and number of purchases for a particular VisitNumber</h3>

In [112]:
df_groups = df.groupby(df['VisitNumber']).groups
len(df_groups.keys())

95674

In [113]:
df_groups

{1: Int64Index([0, 1, 2, 3], dtype='int64'),
 2: Int64Index([4, 5, 6, 7], dtype='int64'),
 3: Int64Index([8, 9], dtype='int64'),
 4: Int64Index([10], dtype='int64'),
 6: Int64Index([11, 12], dtype='int64'),
 13: Int64Index([13, 14], dtype='int64'),
 14: Int64Index([15, 16, 17, 18, 19, 20, 21, 22, 23, 24], dtype='int64'),
 16: Int64Index([25, 26, 27, 28, 29, 30, 31, 32], dtype='int64'),
 18: Int64Index([33, 34, 35, 36, 37, 38, 39], dtype='int64'),
 21: Int64Index([40, 41, 42, 43, 44, 45, 46], dtype='int64'),
 22: Int64Index([47], dtype='int64'),
 24: Int64Index([48], dtype='int64'),
 27: Int64Index([49, 50, 51, 52, 53, 54, 55, 56], dtype='int64'),
 34: Int64Index([57, 58], dtype='int64'),
 35: Int64Index([59], dtype='int64'),
 36: Int64Index([60, 61], dtype='int64'),
 37: Int64Index([62, 63, 64], dtype='int64'),
 38: Int64Index([65], dtype='int64'),
 39: Int64Index([66, 67], dtype='int64'),
 44: Int64Index([68], dtype='int64'),
 46: Int64Index([69, 70], dtype='int64'),
 48: Int64Index([

In [114]:
df_groups = df.groupby(df['VisitNumber']).groups
data = []
for visit_id, ids in df_groups.items():
    #print(dept_desc)
    returns = 0
    purchases = 0
    for idx in ids:
        if df.loc[idx]['ScanCount'] < 0:
            returns += df.loc[idx]['ScanCount']
        else:
            purchases += df.loc[idx]['ScanCount']
    data.append({"VisitNumber":visit_id, "VisitNumReturns":returns, "VisitNumPurchases":purchases})

In [115]:
df2 = pd.DataFrame(data)
df2['VisitNumReturns'] = abs(df2['VisitNumReturns'])
df2.head()

,VisitNumPurchases,VisitNumReturns,VisitNumber
0,4,0,1
1,4,0,2
2,1,1,3
3,1,0,4
4,1,1,6


In [116]:
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,4,0
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,4,0
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,4,0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,4,0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,4,0


<h3>Min, max, mean number of products for a DepartmentDescription</h3>

In [117]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    data.append({"DepartmentDescription":dept_desc, "MinD":min(df['FLCount']), "MaxD":max(df['FLCount']), "MeanD": np.mean(df['FLCount'])})

In [118]:
df2 = pd.DataFrame(data)
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,MaxD,MeanD,MinD
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,4,0,966,258.847552,1
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,4,0,966,258.847552,1
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,4,0,966,258.847552,1
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,4,0,966,258.847552,1
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,4,0,966,258.847552,1


<h3>Min, max, mean number of products for a VisitNumber</h3>

In [119]:
df_groups = df.groupby(df['DepartmentDescription']).groups
data = []
for dept_desc, ids in df_groups.items():
    data.append({"DepartmentDescription":dept_desc, "MinV":min(df['VisitFLCount']), "MaxV":max(df['VisitFLCount']), "MeanV": np.mean(df['VisitFLCount'])})

In [120]:
df2 = pd.DataFrame(data)
df = df.merge(df2, how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,MaxD,MeanD,MinD,MaxV,MeanV,MinV
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,4,0,966,258.847552,1,120,15.171227,1
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,4,0,966,258.847552,1,120,15.171227,1
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,4,0,966,258.847552,1,120,15.171227,1
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,4,0,966,258.847552,1,120,15.171227,1
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,4,0,966,258.847552,1,120,15.171227,1


<h3>Count how many items belonging to each department description are bought in each trip</h3>

In [121]:
departments = df.groupby(df['DepartmentDescription']).groups
departments.keys()

dict_keys(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'MENSWEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 'PHARMACY RX', '

In [122]:
visits_by_group = df.groupby(['VisitNumber','DepartmentDescription']).groups
unq_fl = df.groupby(['VisitNumber','DepartmentDescription'])['FinelineNumber'].nunique()
data = []
for group, ids in visits_by_group.items():
    #print(group[1])
    dept = group[1]
    rept_dept = "REPT_ITEMS_"+dept
    ret_dept = "RETURN_"+dept
    returns = 0
    purchases = 0
    repeats = 0
    for idx in ids:
        if df.loc[idx]['ScanCount'] < 0:
            returns += abs(df.loc[idx]['ScanCount'])
        else:
            purchases += df.loc[idx]['ScanCount']
    if purchases == 0:
        repeats = 0
    else:
        repeats = (purchases-unq_fl[group])
    data.append({"VisitNumber":group[0], dept:purchases, rept_dept:repeats, ret_dept:returns})
    #print(data)

In [123]:
print(data[:5])
len(data)

[{'VisitNumber': 1, 'DAIRY': 2, 'REPT_ITEMS_DAIRY': 0, 'RETURN_DAIRY': 0}, {'VisitNumber': 1, 'GROCERY DRY GOODS': 1, 'REPT_ITEMS_GROCERY DRY GOODS': 0, 'RETURN_GROCERY DRY GOODS': 0}, {'VisitNumber': 1, 'SHOES': 1, 'REPT_ITEMS_SHOES': 0, 'RETURN_SHOES': 0}, {'VisitNumber': 2, 'BAKERY': 1, 'REPT_ITEMS_BAKERY': 0, 'RETURN_BAKERY': 0}, {'VisitNumber': 2, 'DSD GROCERY': 1, 'REPT_ITEMS_DSD GROCERY': 0, 'RETURN_DSD GROCERY': 0}]


322806

In [124]:
#df = df.drop(columns=feature_names)
print(df.columns)

Index(['VisitNumber', 'Weekday', 'Upc', 'ScanCount', 'DepartmentDescription',
       'FinelineNumber', 'FLCount', 'VisitFLCount', 'NumPurchases',
       'NumReturns', 'VisitNumPurchases', 'VisitNumReturns', 'MaxD', 'MeanD',
       'MinD', 'MaxV', 'MeanV', 'MinV'],
      dtype='object')


In [125]:
#print("REPT_ITEMS_"+list(df_groups_train.keys())[0])
feature_names = [d for d in list(df_groups_train.keys())]
feature_names.extend(["REPT_ITEMS_"+d for d in list(df_groups_train.keys())])
feature_names.extend(["RETURN_"+d for d in list(df_groups_train.keys())])
feature_names.append("VisitNumber")
print(len(feature_names))
df2 = pd.DataFrame(columns=feature_names).set_index("VisitNumber")
df2 = df2.from_records(data, columns=feature_names).fillna(0).set_index("VisitNumber")
df2 = df2.groupby("VisitNumber").sum()
df2.head()

205


,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,RETURN_SEAFOOD,RETURN_SEASONAL,RETURN_SERVICE DELI,RETURN_SHEER HOSIERY,RETURN_SHOES,RETURN_SLEEPWEAR/FOUNDATIONS,RETURN_SPORTING GOODS,RETURN_SWIMWEAR/OUTERWEAR,RETURN_TOYS,RETURN_WIRELESS
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
df["VisitNumber"].nunique()

95674

In [127]:
df = df.join(df2, on='VisitNumber', how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,...,RETURN_SEAFOOD,RETURN_SEASONAL,RETURN_SERVICE DELI,RETURN_SHEER HOSIERY,RETURN_SHOES,RETURN_SLEEPWEAR/FOUNDATIONS,RETURN_SPORTING GOODS,RETURN_SWIMWEAR/OUTERWEAR,RETURN_TOYS,RETURN_WIRELESS
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
df_groups = df.groupby(df['VisitNumber']).groups
data = []
for visit, ids in df_groups.items():
    #print(dept_desc)
    flist = []
    for idx in ids:
        flist.append(df.loc[idx]['FLCount'])
    data.append({"VisitNumber":visit, "MinFLCount":np.min(flist), "MaxFLCount":np.max(flist), "StdFLCount":np.std(flist) })

In [129]:
data[:5]

[{'MaxFLCount': 464,
  'MinFLCount': 108,
  'StdFLCount': 164.17273677441088,
  'VisitNumber': 1},
 {'MaxFLCount': 362,
  'MinFLCount': 93,
  'StdFLCount': 111.26629094204587,
  'VisitNumber': 2},
 {'MaxFLCount': 305, 'MinFLCount': 305, 'StdFLCount': 0.0, 'VisitNumber': 3},
 {'MaxFLCount': 340, 'MinFLCount': 340, 'StdFLCount': 0.0, 'VisitNumber': 4},
 {'MaxFLCount': 179, 'MinFLCount': 179, 'StdFLCount': 0.0, 'VisitNumber': 6}]

In [130]:
df2 = pd.DataFrame(data).fillna(0).set_index("VisitNumber")
df = df.join(df2, on='VisitNumber', how='left')
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,...,RETURN_SHEER HOSIERY,RETURN_SHOES,RETURN_SLEEPWEAR/FOUNDATIONS,RETURN_SPORTING GOODS,RETURN_SWIMWEAR/OUTERWEAR,RETURN_TOYS,RETURN_WIRELESS,MaxFLCount,MinFLCount,StdFLCount
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464,108,164.172737
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464,108,164.172737
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464,108,164.172737
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,464,108,164.172737
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362,93,111.266291


<h3>Top 200 frequent UPC and Fineline numbers featuers</h3>

In [131]:
data = []
for visit, ids in df_groups.items():
    for idx in ids:
        upc_count = 0
        fl_count = 0
        if df.loc[idx]['Upc'] in freq_upc:
            upc_count = 1
            feat1 = "UPC_"+str(df.loc[idx]['Upc'])
            feat2 = "INDICATOR_UPC_"+str(df.loc[idx]['Upc'])
            data.append({"VisitNumber":visit, feat1:upc_count, feat2:1})
        if df.loc[idx]['FinelineNumber'] in freq_fineline:
            fl_count = 1
            feat1 = "FL_"+str(df.loc[idx]['FinelineNumber'])
            feat2 = "INDICATOR_FL_"+str(df.loc[idx]['FinelineNumber'])
            data.append({"VisitNumber":visit, feat1:fl_count, feat2:1})

In [132]:
data[:5]

[{'FL_3555.0': 1, 'INDICATOR_FL_3555.0': 1, 'VisitNumber': 1},
 {'FL_8023.0': 1, 'INDICATOR_FL_8023.0': 1, 'VisitNumber': 2},
 {'FL_8023.0': 1, 'INDICATOR_FL_8023.0': 1, 'VisitNumber': 2},
 {'FL_4606.0': 1, 'INDICATOR_FL_4606.0': 1, 'VisitNumber': 13},
 {'FL_3658.0': 1, 'INDICATOR_FL_3658.0': 1, 'VisitNumber': 14}]

In [133]:
df2 = pd.DataFrame(data).fillna(0).set_index("VisitNumber")
indicators = ["INDICATOR_FL_"+str(fl) for fl in freq_fineline]
indicators.extend(["INDICATOR_UPC_"+str(upc) for upc in freq_upc])
df3 = df2.groupby("VisitNumber").sum()
df3[indicators] = df3[indicators].astype(bool).astype(int)
df3 = df3.fillna(0)
df3.head()

,FL_0.0,FL_100.0,FL_1000.0,FL_1001.0,FL_1006.0,FL_1008.0,FL_1026.0,FL_103.0,FL_104.0,FL_109.0,...,UPC_7874298393.0,UPC_7874298761.0,UPC_7976503128.0,UPC_8265750406.0,UPC_83032400641.0,UPC_87458603436.0,UPC_88828940068.0,UPC_9506.0,UPC_9518801128.0,UPC_980012301.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
df = df.join(df3, on='VisitNumber', how='left')
df = df.fillna(0)
df.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,...,UPC_7874298393.0,UPC_7874298761.0,UPC_7976503128.0,UPC_8265750406.0,UPC_83032400641.0,UPC_87458603436.0,UPC_88828940068.0,UPC_9506.0,UPC_9518801128.0,UPC_980012301.0
0,1,4,7.250339e+10,1,SHOES,3002.0,464,4,6131,292,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,1.707711e+09,1,DAIRY,1526.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,4,8.947000e+10,1,DAIRY,1431.0,108,4,52272,415,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,403,4,92757,874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,362,3,81199,848,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
#df[df['VisitNumber'] == 8]['StdFLCount']

In [136]:
df2 = df.groupby("VisitNumber").mean()

In [137]:

#df[df['VisitNumber'] == 8].to_csv('check.csv')

In [138]:
df_final = df.groupby("VisitNumber").mean()
df_final.head()

,Weekday,Upc,ScanCount,FinelineNumber,FLCount,VisitFLCount,NumPurchases,NumReturns,VisitNumPurchases,VisitNumReturns,...,UPC_7874298393.0,UPC_7874298761.0,UPC_7976503128.0,UPC_8265750406.0,UPC_83032400641.0,UPC_87458603436.0,UPC_88828940068.0,UPC_9506.0,UPC_9518801128.0,UPC_980012301.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,4.0,6.304308e+10,1.0,2378.50,270.75,4.0,50858.0,499.00,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,4.640786e+10,1.0,6368.25,169.75,3.0,38357.0,489.75,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,7.410811e+09,0.0,4504.00,305.00,1.0,44976.0,813.00,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,7.287926e+09,1.0,5924.00,340.00,1.0,5992.0,272.00,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,7.603139e+09,0.0,654.00,179.00,1.0,3952.0,167.00,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h3>Convert to CSV</h3>

In [139]:
df_final.isnull().sum()

Weekday                    0
Upc                        0
ScanCount                  0
FinelineNumber             0
FLCount                    0
VisitFLCount               0
NumPurchases               0
NumReturns                 0
VisitNumPurchases          0
VisitNumReturns            0
MaxD                       0
MeanD                      0
MinD                       0
MaxV                       0
MeanV                      0
MinV                       0
1-HR PHOTO                 0
ACCESSORIES                0
AUTOMOTIVE                 0
BAKERY                     0
BATH AND SHOWER            0
BEAUTY                     0
BEDDING                    0
BOOKS AND MAGAZINES        0
BOYS WEAR                  0
BRAS & SHAPEWEAR           0
CAMERAS AND SUPPLIES       0
CANDY, TOBACCO, COOKIES    0
CELEBRATION                0
COMM BREAD                 0
                          ..
UPC_7874203951.0           0
UPC_7874203952.0           0
UPC_7874204025.0           0
UPC_7874204026

In [140]:
df_final.to_csv(processed_fname)

In [141]:
print(processed_fname)

data/processed_data/processed_test.csv
